# **Celtic Mutation**


![sentiment.png](https://i.ibb.co/VCYKQ2z/nlpmod22.png)

> The code here are divided in 2 parts


*   Done from scratch
*   Used different algorithms to compare results






---




*   **Scratch** 
    * We will use Viterbi Algorithm and will also try a modified version of it.

*   **Anything Goes**
    * In this we will use pre isntalled library NLTK. 
      1.   UnigramTagger
      2.   BigramTagger
      3.   TrigramTagger
      4.   HiddenMarkovModelTrainer





In [63]:
# Importing libraries

import nltk, re
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from IPython.display import HTML, display


In [2]:
'''
        Data Loading:
        ----------
        The data is loaded from a tsv file
        ---------

        Creating Sentences: 
        ----------
        For tags the sentences are created using <S> to define start of a sentence
        ----------
'''
whole_text = []

def tagSetupTrain(): 
  testfile = open('train.tsv', 'r')
  sentence = []
  for line in testfile:
    pieces = line.rstrip("\n").split("\t")
    if pieces[0]=='<S>':
      whole_text.append((sentence))
      sentence = []
    else:
      sentence.append(tuple(pieces))

In [3]:
tagSetupTrain()

In [ ]:
# whole_text = whole_text[:50000]

In [4]:
# split data into training and validation set in the ratio 80:20

random.seed(1234)
train_set,test_set = train_test_split(whole_text,test_size=0.2,random_state = 101)

print("-" * 100)
print("Training Set Length -", len(train_set))
print("Testing Set Length -", len(test_set))
print("-" * 100)
print("Training Data Glimpse -\n")
print(train_set[:1])
print("-" * 100)

----------------------------------------------------------------------------------------------------
Training Set Length - 316737
Testing Set Length - 79185
----------------------------------------------------------------------------------------------------
Training Data Glimpse -

[[('is', 'N'), ('féidir', 'N'), ('leis', 'N'), ('an', 'N'), ('múinteoir', 'N'), ('scileanna', 'N'), ('éisteachta', 'N'), ('an', 'N'), ('ranga', 'N'), ('a', 'N'), ('measúnú', 'S'), ('•', 'N'), ('•', 'N'), ('•', 'N'), ('nuair', 'N'), ('a', 'N'), ('bíonn', 'S'), ('teagasc', 'N'), ('foirmiúil', 'N'), ('teanga', 'N'), ('ar', 'N'), ('siúl', 'N'), ('tríd', 'N'), ('an', 'N'), ('caoi', 'U'), ('a', 'N'), ('freagraíonn', 'U'), ('an', 'N'), ('páiste', 'N'), ("d'úsáid", 'N'), ('spontáineach', 'N'), ('teanga', 'N'), ('teagmhasaí', 'N'), ('i', 'N'), ('rith', 'N'), ('an', 'N'), ('lá', 'N'), ('scoile', 'N'), ('a', 'N'), ('breathnú', 'S'), ('trí', 'N'), ('ceisteanna', 'S'), ('a', 'N'), ('cur', 'S'), ('ar', 'N'), ('an', 'N'), 

In [5]:
# create list of train and test tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
print(len(train_tagged_words))
print(len(test_tagged_words))

7686253
1917825


In [6]:
# check some of the tagged words.
train_tagged_words[1:5]

[('féidir', 'N'), ('leis', 'N'), ('an', 'N'), ('múinteoir', 'N')]

In [7]:
# let's check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)

5
{'T', 'N', 'U', 'S', 'H'}


In [8]:
# let's check how many words are present in vocabulary
vocab = {word for word,tag in train_tagged_words}
print(len(vocab))

151011


**POS Tagging algorithm using Hidden Markov Model (HMM)**

We'll use the HMM algorithm to tag the words. Given a sequence of words to be tagged, the task is to assign the most probable tag to the word. In other words, to every word w, assign the tag t that maximises the likelihood P(t/w).


Since P(t/w) = P(w/t). P(t) / P(w), after ignoring P(w), we have to compute P(w/t) and P(t).

Now:



*   **P(w/t): is the emission probability** of a given word for a given tag. This can be computed based on the fraction of given word for given tag to the total count of that tag, ie: P(w/t) = count(w, t) / count(t).


*   **P(t): is the probability of tag (also transition probability)** , and in a tagging task, we assume that a tag will depend only on the previous tag (Markov order 1 assumption). In other words, the probability of say a tag being NN will depend only on the previous tag t(n-1). So for e.g. if t(n-1) is a JJ, then t(n) is likely to be an NN since adjectives often precede a noun (blue coat, tall building etc.).





In [20]:
# compute emission probability for a given word for a given tag
def word_given_tag(word,tag,train_bag= train_tagged_words):
    """"
        Parameters:
        ----------
        word: individualw word w
        train_bag: it is the training set that we initialized at top.
        
        What the function does?
        -----------------------
        It computes emission probabilties for a given word.
        
    """
    taglist = [pair for pair in train_bag if pair[1] == tag]
    tag_count = len(taglist)    
    w_in_tag = [pair[0] for pair in taglist if pair[0]==word]    
    word_count_given_tag = len(w_in_tag)    
    
    return (word_count_given_tag,tag_count)

In [24]:
# compute transition probabilities of a previous and next tag
def t2_given_t1(t2,t1,train_bag=train_tagged_words):
    """"
        Parameters:
        ----------
        t2: tag
        t1: tag
        train_bag: it is the training set that we initialized at top.
        
        What the function does?
        -----------------------
        It ompute transition probabilities of a previous and next tag
        
    """

    tags = [pair[1] for pair in train_bag]
    t1_tags = [tag for tag in tags if tag==t1]
    count_of_t1 = len(t1_tags)
    t2_given_t1 = [tags[index+1] for index in range(len(tags)-1) if tags[index] == t1 and tags[index+1] == t2]
    count_t2_given_t1 = len(t2_given_t1)
    return(count_t2_given_t1,count_of_t1)

In [28]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [30]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))

# dataset glimpse
tags_df

,T,N,U,S,H
T,0.000144,0.939395,0.000072,0.057376,0.003014
N,0.004241,0.835945,0.039722,0.110549,0.009542
U,0.000050,0.960100,0.000688,0.038504,0.000658
S,0.000064,0.944772,0.001225,0.052296,0.001642
H,0.000187,0.956280,0.000887,0.040170,0.002476


In [26]:
# plt.figure(figsize=(14, 8))
# sns.heatmap(tags_df, annot = True)
# plt.show()

In [27]:
# tags_frequent = tags_df[tags_df>0.5]
# plt.figure(figsize=(14, 8))
# sns.heatmap(tags_frequent, annot = True)
# plt.show()

Accoridng to **Viterbi Algorithm**



1.   Given a sequence of words
2.   iterate through the sequence
3.    for each word (starting from first word in sequence) calculate the product of emission probabilties and transition probabilties for all possible tags.
4. assign the tag which has maximum probability obtained in step 3 above.
5. move to the next word in sequence to repeat steps 3 and 4 above.





In [31]:
# Vanilla Viterbi Algorithm
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = 0
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [32]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [45]:
# tagging the test sentences
tagged_seq = Viterbi(test_tagged_words)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

accuracy = len(check)/len(tagged_seq)
print('Vanilla Viterbi Algorithm Accuracy: ',accuracy*100)
acc_1 = accuracy*100

Vanilla Viterbi Algorithm Accuracy:  88.110189451862


In [34]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('gaoth', 'T'), ('gaoth', 'N')),
 (('tí', 'U'), ('tí', 'N')),
 (('freagra', 'N'), ('freagra', 'S')),
 (('aye', 'N'), ('aye', 'H')),
 (('boente', 'T'), ('boente', 'N')),
 (('aye', 'N'), ('aye', 'H')),
 (('troid', 'N'), ('troid', 'S')),
 (('diaidh', 'U'), ('diaidh', 'S')),
 (('tar', 'N'), ('tar', 'S')),
 (('cóta', 'N'), ('cóta', 'S')),
 (('craoladh', 'N'), ('craoladh', 'S')),
 (('tar', 'N'), ('tar', 'S')),
 (('creata', 'S'), ('creata', 'N')),
 (('giuirléidí', 'N'), ('giuirléidí', 'S')),
 (('aistriú', 'N'), ('aistriú', 'U')),
 (('tar', 'N'), ('tar', 'S'))]

**Viterbi Version 2**

Use transition probability of tags when emission probability is zero (in case of unknown words)

In [36]:
def Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # list for storing transition probabilities
        for tag in T:
            if key == 0:
                transition_p = 0
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # if probability is zero (unknown word) then use transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [39]:
tagged_seq_v1 = Viterbi_1(test_tagged_words)


# accuracy
check_v1 = [i for i, j in zip(tagged_seq_v1, test_run_base) if i == j] 
accuracy_v1 = len(check_v1)/len(tagged_seq_v1)
print('Modified Viterbi_1 Accuracy: ',accuracy_v1*100)

Modified Viterbi_1 Accuracy:  89.1891891891892


In [44]:
# tagged_seq

**Adding Tag occurance probability weights**

 we will apply weights based on the probability of tag occurance to the transition probabilities of tags and then use the resulting probability for predicting unknown words.

This scheme will also take into account that some POS tags are more likely to occur as compared to others.

In [40]:
# lets create a list containing tuples of POS tags and POS tag occurance probability, based on training data
tag_prob = []
total_tag = len([tag for word,tag in train_tagged_words])
for t in tags:
    each_tag = [tag for word,tag in train_tagged_words if tag==t]
    tag_prob.append((t,len(each_tag)/total_tag))

tag_prob

[('T', 0.003625823922267456),
 ('N', 0.85258616909956),
 ('U', 0.034022039087185915),
 ('S', 0.10140988073122235),
 ('H', 0.008356087159764321)]

In [42]:
def Viterbi_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # list for storing transition probabilities
       
        for tag in T:
            if key == 0:
                transition_p = 0
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
            # find POS tag occurance probability
            tag_p = [pair[1] for pair in tag_prob if pair[0]==tag ]
            
            # calculate the transition prob weighted by tag occurance probability.
            transition_p = tag_p[0]*transition_p             
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # if probability is zero (unknown word) then use weighted transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]                 
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [58]:

tagged_seq_v2 = Viterbi_2(test_tagged_words)


# accuracy
check_v2 = [i for i, j in zip(tagged_seq_v2, test_run_base) if i == j] 
accuracy_v2 = len(check)/len(tagged_seq_v2)
print('Modified Viterbi_1 Accuracy: ',accuracy_v2*100)
acc_2 = accuracy_v2*100


Modified Viterbi_1 Accuracy:  89.1891891891892


In [47]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq_v2, test_run_base)) if j[0] != j[1]]

[(('gaoth', 'T'), ('gaoth', 'N')),
 (('tí', 'U'), ('tí', 'N')),
 (('freagra', 'N'), ('freagra', 'S')),
 (('aye', 'N'), ('aye', 'H')),
 (('aye', 'N'), ('aye', 'H')),
 (('troid', 'N'), ('troid', 'S')),
 (('diaidh', 'U'), ('diaidh', 'S')),
 (('tar', 'N'), ('tar', 'S')),
 (('cóta', 'N'), ('cóta', 'S')),
 (('craoladh', 'N'), ('craoladh', 'S')),
 (('tar', 'N'), ('tar', 'S')),
 (('creata', 'S'), ('creata', 'N')),
 (('giuirléidí', 'N'), ('giuirléidí', 'S')),
 (('aistriú', 'N'), ('aistriú', 'U')),
 (('tar', 'N'), ('tar', 'S'))]

# **Anything Goes**

* In this we will use pre isntalled library NLTK. 
      1.   UnigramTagger
      2.   BigramTagger
      3.   TrigramTagger
      4.   HiddenMarkovModelTrainer

In [48]:
import nltk   #import required libraries
from nltk.tag import hmm
from nltk.stem import PorterStemmer
from nltk.corpus.reader import TaggedCorpusReader 
from nltk.corpus.reader import PlaintextCorpusReader
from sklearn import metrics  #for evaluation purposes


In [49]:
textAuto = []
tagdT = []
def tagAlign(): 
  testfile = open('train.tsv', 'r')
  sentS = []
  for line in testfile:
#     total += 1
    pieces = line.rstrip("\n").split("\t")
    if pieces[0]=='<S>':
      textAuto.append((sentS))
      sentS = []
    else:
      sentS.append(tuple(pieces))
      tagdT.append(tuple(pieces))

tagAlign()

In [50]:
#frequency distribution of tags in our data
tag_fd = nltk.FreqDist(tag for (word, tag) in tagdT)
tag_fd.most_common(5)

[('N', 8188100), ('S', 973469), ('U', 327110), ('H', 80504), ('T', 34895)]

In [51]:
# divide the data into train and dev set with ratio of split = 10%
split = int(len(whole_text) * 0.9)
split

356329

In [52]:
train_sents = whole_text[:split]
dev_sents = whole_text[split:]

In [53]:
#Training and testing unigram tagger
unigram_tagger = nltk.UnigramTagger(train_sents)  
acc_3 = unigram_tagger.evaluate(dev_sents)*100
print("Unigram accuracy:", acc_3)

Unigram accuracy: 89.54418361352649


In [54]:
#Using backoff tagger for bi-tri tagger
bigram_tagger = nltk.BigramTagger(train_sents, backoff=unigram_tagger)  
acc_4 = bigram_tagger.evaluate(dev_sents)*100
print("Bigram accuracy:",acc_4)


trigram_tagger = nltk.TrigramTagger(train_sents, backoff=bigram_tagger)
acc_5 = trigram_tagger.evaluate(dev_sents)*100
print("Trigram accuracy:",acc_5)


Bigram accuracy: 89.88543874325117
Trigram accuracy: 89.96265705727647


In [56]:
Training and tagging HMM using nltk trainer
hmm_trainer = hmm.HiddenMarkovModelTrainer()
hmm_tagger = hmm_trainer.train_supervised(train_sents)
print("HMM train accuracy:", hmm_tagger.evaluate(train_sents)*100)
acc_6 = hmm_tagger.evaluate(dev_sents)*100
print("HMM dev accuracy:", acc_6)

# print("HMM train accuracy:", '91.26658395451258')
# print("HMM dev accuracy:", '89.99008236553826')

HMM train accuracy: 91.26658395451258
HMM dev accuracy: 89.99008236553826


In [67]:
#Printing everything
def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4><td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

data = [['Viterbi V1 :',acc_1],['Viterbi V2 : ',acc_2],['UnigramTagger :',acc_3],['BigramTagger :',acc_4],['TrigramTagger :',acc_5],['HiddenMarkovModelTrainer :',acc_6]]
display_table(data)


**To test new data just replace the file path and rest will work.**

In [ ]:
whole_test_text = []

def evaluate():
    testfile = open('train.tsv', 'r')
    sentence_test = []
    for line in testfile:
        pieces = line.rstrip("\n").split("\t")
        if pieces[0]=='<S>':
          whole_test_text.append((sentence_test))
          sentence_test = []
        else:
          sentence_test.append(tuple(pieces))
    
#     test = whole_test_text[50000:55000]
#     random.seed(1234)
#     rndom = [random.randint(1,len(whole_test_text)) for x in range(5)]
#     test_run = [test[i] for i in rndom]
    test_run_base = [tup for sent in whole_test_text for tup in sent]
    test_tagged_words = [tup[0] for sent in whole_test_text for tup in sent]
    
    tagged_seq1 = Viterbi_1(test_tagged_words)
    check1 = [i for i, j in zip(tagged_seq1, test_run_base) if i == j] 
    accuracy1 = len(check1)/len(tagged_seq1)
    print('Modified Viterbi_1 Accuracy: ',accuracy1*100)
    
    tagged_seq2 = Viterbi_2(test_tagged_words)
    check2 = [i for i, j in zip(tagged_seq2, test_run_base) if i == j] 
    accuracy2 = len(check2)/len(tagged_seq2)
    print('Modified Viterbi_1 Accuracy: ',accuracy2*100)
    


In [ ]:
evaluate()

# **Acknowledgement :**



*  [Dr. Kevin Scannell : Neural Models for Predicting Celtic Mutations ](https://www.aclweb.org/anthology/2020.sltu-1.1.pdf)
*   [Medium](https://medium.com/data-science-in-your-pocket/pos-tagging-using-hidden-markov-models-hmm-viterbi-algorithm-in-nlp-mathematics-explained-d43ca89347c4)
* [Towards Data Science](https://towardsdatascience.com/part-of-speech-tagging-with-hidden-markov-chain-models-e9fccc835c0e)

